↓ ↓ ↓ A Chronicle of Early Failures ↓ ↓ ↓

In [1]:
!apt-get update
!apt-get install -y chromium-chromedriver
!pip install selenium
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install openpyxl

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,089 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,889 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [53.5 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [51.5 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,390 kB]
Hit:12 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:13 https://ppa.launchpadconten

In [ ]:
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import Workbook
from openpyxl.styles import Alignment, Font, PatternFill
import time
from tqdm import tqdm

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument('--window-size=1920x1080')

# Initialize the WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Function to scroll and collect comments
def scroll_and_collect_comments(driver, video_url):
    # Initialize comments array with video URL at the top
    comments = np.array([])

    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

    # Wait for new comments to load
    time.sleep(5)

    # Extract comments
    comment_elements = driver.find_elements(By.CSS_SELECTOR, "#content-text")
    new_comments = np.array([element.text for element in comment_elements])  # Convert comments to NumPy array
    comments = np.concatenate((comments, new_comments))  # Append new comments to existing array
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    comments = np.unique(comments)

    return comments

# Function to navigate to a YouTube video and extract comments
def get_youtube_comments(video_url):
    driver.get(video_url)

    # Wait for the page to load completely
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    time.sleep(5)  # Additional wait to ensure comments section loads

    # Scroll and collect comments
    comments = scroll_and_collect_comments(driver, video_url)

    return comments

# Function to save comments to XLSX with formatting
def save_comments_to_xlsx(video_url, comments, worksheet):
    # Write video URL in the first cell
    worksheet.cell(row=worksheet.max_row + 1, column=1).value = video_url
    worksheet.cell(row=worksheet.max_row, column=1).font = Font(bold=True)  # Highlight the header
    worksheet.cell(row=worksheet.max_row, column=1).alignment = Alignment(horizontal='center')

    # Write comments
    for comment in comments:
        worksheet.cell(row=worksheet.max_row + 1, column=2).value = comment
        worksheet.cell(row=worksheet.max_row, column=2).alignment = Alignment(wrap_text=True)

    # Fill color for both columns
    fill = PatternFill(start_color="D9D9D9", end_color="D9D9D9", fill_type="solid")
    for row_num in np.arange(worksheet.max_row - len(comments), worksheet.max_row + 1):
        for col_num in [1, 2]:
            worksheet.cell(row=row_num, column=col_num).fill = fill

# Function to save progress to Excel file
def save_progress_to_excel(filename, worksheet):
    worksheet.sheet_view.showGridLines = False  # Hide gridlines
    for column in worksheet.columns:
        max_length = max(len(str(cell.value)) for cell in column)
        adjusted_width = (max_length + 2) * 1.2
        worksheet.column_dimensions[column[0].column_letter].width = adjusted_width
    worksheet.freeze_panes = 'A2'  # Freeze header row
    workbook.save(filename)

# Prompt for YouTube video URLs
video_urls = [
    "https://youtu.be/e_A2b7azy5M?si=WFwYvCw7rUlQB-yo",
"https://youtu.be/hSud7Q__SXk?si=4wR4Vdn8Gtb05WLh",
"https://www.youtube.com/watch?v=Ih9oyP9-nSg&t=24s",
"https://www.youtube.com/watch?v=DM6VZaprRvQ",
"https://www.youtube.com/watch?v=8qn_qfSk9mY",
"https://www.youtube.com/watch?v=gVRv09dOz4g&t=16s",
"https://www.youtube.com/watch?v=EXIjU6lIfhg&t=2s",
"https://www.youtube.com/watch?v=cw4jxvN5_b0",
"https://www.youtube.com/watch?v=Z4rxkg1SD0M",
"https://www.youtube.com/watch?v=y0LBEiJdFVo",
"https://www.youtube.com/watch?v=D6nM-YihR1M",
"https://www.youtube.com/watch?v=-gjDW4yPaTc",
"https://www.youtube.com/watch?v=wM-UdgU2gyM",
"https://www.youtube.com/watch?v=52AhTdlfw9A",
"https://www.youtube.com/watch?v=JUBIv3m-y2g",
"https://www.youtube.com/watch?v=uOVUj_eLB3g",
"https://www.youtube.com/watch?v=VrRGZ6S5bAM",
"https://www.youtube.com/watch?v=_x0yLcfPRtM",
"https://www.youtube.com/watch?v=jIcdEmOpMBU",
"https://www.youtube.com/watch?v=CeYggs_XQfM",
"https://www.youtube.com/watch?v=oAkcpomWLmc",
"https://www.youtube.com/watch?v=wGHcYkIhXZY",
"https://www.youtube.com/watch?v=tJ6xIcnhWr8",
"https://www.youtube.com/watch?v=GcvtcRgrorw",
"https://www.youtube.com/watch?v=rJrAweahkzU",
"https://www.youtube.com/watch?v=R2z2jpj9pDg",
"https://www.youtube.com/watch?v=P5I_qn4FJ54",
"https://www.youtube.com/watch?v=k3nr1QpT5qk",
"https://www.youtube.com/watch?v=Am8W30Y9rkc",
"https://www.youtube.com/watch?v=TOQp6ap_xS8",
"https://www.youtube.com/watch?v=zmB4LuA1u3I",
"https://www.youtube.com/watch?v=DpBS8X2EeJM",
"https://www.youtube.com/watch?v=fasAF5n5O_g",
"https://www.youtube.com/watch?v=jUMuLBaANsI",
"https://www.youtube.com/watch?v=rRF7V6Dc1zE",
"https://www.youtube.com/watch?v=ngx-8AlUGLg",
"https://www.youtube.com/watch?v=zfPITjBQcEc",
"https://www.youtube.com/watch?v=dtThn6AplI0",
"https://www.youtube.com/watch?v=-6KXFIJ8RvY",
"https://www.youtube.com/watch?v=ci4tliK2iAI",
"https://www.youtube.com/watch?v=VCnGWqau9X4&t=1s",
"https://www.youtube.com/watch?v=qzNEN_y-IMI&t=36s",
"https://www.youtube.com/watch?v=_XEWGvENx4I",
"https://www.youtube.com/watch?v=YyHPo4BWx6U",
"https://www.youtube.com/watch?v=N4u5GiwsOwA",
"https://www.youtube.com/watch?v=VVSZ0RERHiQ",
"https://www.youtube.com/watch?v=UV0s668MuLI",
"https://www.youtube.com/watch?v=uU8-W9NQDs4",
"https://www.youtube.com/watch?v=DEbBj8rav0s",
"https://www.youtube.com/watch?v=QdxJEW6OBD0",
"https://www.youtube.com/watch?v=OT2hoiGQG2E",
"https://www.youtube.com/watch?v=e6daGC1py-s",
"https://www.youtube.com/watch?v=aIxhheHGMq0",
"https://www.youtube.com/watch?v=OE83-DLcWCs",
"https://www.youtube.com/watch?v=pUvgSAL9VE8",
"https://www.youtube.com/shorts/95UDV1EJwpI",
"https://www.youtube.com/watch?v=So8gZKlns5I",
"https://www.youtube.com/shorts/Dhj49SJLEIM",
"https://www.youtube.com/watch?v=MdR4M5HO0Ks",
"https://www.youtube.com/watch?v=kRGEDBsLHkE",
"https://www.youtube.com/shorts/cmmsIL3LSKw",
"https://www.youtube.com/watch?v=21vtQq7okQ4",
"https://www.youtube.com/watch?v=_-QvM3fX41c",
"https://www.youtube.com/watch?v=3BS7gpZ7WDU",
"https://www.youtube.com/watch?v=uwf8iERIK7c",
"https://www.youtube.com/watch?v=cJXbg0Mmwp8",
"https://www.youtube.com/watch?v=e_OFK7kzCcY",
"https://www.youtube.com/watch?v=4LO1Ib3TunA",
"https://www.youtube.com/watch?v=2YR8lwNB-no",
"https://www.youtube.com/watch?v=U2qV-sArxcs",
"https://www.youtube.com/watch?v=5MTy9E6XIJs",
"https://www.youtube.com/watch?v=k-tp0NcunzM",
"https://www.youtube.com/watch?v=a6BjaOGpj5E",
"https://www.youtube.com/watch?v=Ltr1xifKr30",
"https://www.youtube.com/watch?v=8voZ4SJXxJc",
"https://www.youtube.com/watch?v=rQo8KTfXsrA",
"https://www.youtube.com/watch?v=ibnhEite9H8&t=13s",
"https://www.youtube.com/watch?v=pCwuuREcGOY",
"https://www.youtube.com/watch?v=uS_SO1xKXKc&t=133s",
"https://www.youtube.com/watch?v=N1GJ-qjnLRw",
"https://www.youtube.com/watch?v=Xie2ZLksSJs&t=21s",
"https://www.youtube.com/watch?v=usawEwn9mL8&t=102s",
"https://www.youtube.com/watch?v=Y0wRJCcXwIY",
"https://www.youtube.com/watch?v=EKM9mJygKg4",
"https://www.youtube.com/watch?v=2PEohKk4bos",
"https://www.youtube.com/watch?v=5lRSuHbTNgg",
"https://www.youtube.com/watch?v=bKU9w3pYEg0",
"https://www.youtube.com/watch?v=vDsKdysqneo",
"https://www.youtube.com/watch?v=RU1wNYuq3FU",
"https://www.youtube.com/watch?v=vE_eqgSD56U",
"https://www.youtube.com/watch?v=yUcdRywNDb4",
"https://www.youtube.com/watch?v=BrO_pjgpUK4",
"https://www.youtube.com/watch?v=KUGg0Kz4KGI",
"https://www.youtube.com/watch?v=VNovwNMhnkk",
"https://www.youtube.com/watch?v=BRKyJf7qzq0",
"https://www.youtube.com/watch?v=z7k9qADcsrs",
"https://www.youtube.com/watch?v=OjIR-rRqXc0",
"https://www.youtube.com/watch?v=WTzITFKOno0",
"https://www.youtube.com/watch?v=_8zHWqKNSHA",
"https://www.youtube.com/watch?v=T_h9iEzoyjQ",
"https://www.youtube.com/watch?v=43rhNDSg7qU",
"https://www.youtube.com/watch?v=h5VLU_0_ZyA",
"https://www.youtube.com/watch?v=PFT-XEGDqCs",
"https://www.youtube.com/watch?v=cA1Nhzk73ZQ",
"https://www.youtube.com/watch?v=Op2oaWTfNZE",
"https://www.youtube.com/watch?v=Q-QcXlZbhm4",
"https://www.youtube.com/watch?v=6Z0O2fx3abA",
"https://www.youtube.com/watch?v=Me7uFvDknTE",
"https://www.youtube.com/watch?v=UiMJHDIkgU0",
"https://www.youtube.com/watch?v=XCcyVZgM7u4",
"https://www.youtube.com/watch?v=mTYdMuBOR74",
"https://www.youtube.com/watch?v=CeL4oL8_2og",
"https://www.youtube.com/watch?v=rYdHF3AEiHw",
"https://www.youtube.com/watch?v=Y9L5NmFsR6U",
"https://www.youtube.com/watch?v=2meaYegHBQE",
"https://www.youtube.com/watch?v=431ncfs1HLQ",
"https://www.youtube.com/watch?v=hGdP_MBIqk8",
"https://www.youtube.com/watch?v=DWvSdnPrndY",
"https://www.youtube.com/watch?v=XnwU2D8LojY",
"https://www.youtube.com/watch?v=fCGwtMbKYu8",
"https://www.youtube.com/watch?v=vJkz8c2_niE",
"https://www.youtube.com/watch?v=gC1Dy2n0wj8",
"https://www.youtube.com/watch?v=lGXHVYKsIeg",
"https://www.youtube.com/watch?v=0sn_UUZnyKk",
"https://www.youtube.com/watch?v=8-CPA1OAmgA",
"https://www.youtube.com/watch?v=2ILIXOFvbXI",
"https://www.youtube.com/watch?v=7nwH1bEThWE",
"https://www.youtube.com/watch?v=muMxUjTuqGw",
"https://www.youtube.com/watch?v=r4mySrqyyEI",
"https://www.youtube.com/watch?v=GsL40UZjw_E",
"https://www.youtube.com/watch?v=0M-s8ND62D8",
"https://www.youtube.com/watch?v=jJx877-QYRo",
"https://www.youtube.com/watch?v=jJx877-QYRo",
"https://www.youtube.com/watch?v=gPY9l9PBDrI",
"https://www.youtube.com/watch?v=KFm3oKGyz4k",
"https://www.youtube.com/watch?v=0M-s8ND62D8",
"https://www.youtube.com/watch?v=-T6D1QtaXLg",
]

# Create a new workbook
workbook = Workbook()
worksheet = workbook.active

# Set header row
worksheet['A1'] = 'Video URL'
worksheet['B1'] = 'Comment'

# Apply formatting to header row
header_font = Font(bold=True)
header_fill = PatternFill(start_color="A6A6A6", end_color="A6A6A6", fill_type="solid")
for cell in worksheet["1:1"]:
    cell.font = header_font
    cell.fill = header_fill

# Process each video URL
for video_url in tqdm(video_urls, desc="Processing videos"):
    if video_url:
        comments = get_youtube_comments(video_url)
        save_comments_to_xlsx(video_url, comments, worksheet)
        save_progress_to_excel('yt_comments_in_progress.xlsx', worksheet)

# Save the final workbook
workbook.save('yt_comments.xlsx')

# Download the file
from google.colab import files
files.download('yt_comments.xlsx')


Processing videos:  76%|███████▌  | 104/137 [24:30<07:49, 14.24s/it]